In [1]:
import requests
import datetime as dt
import pandas as pd
from datetime import datetime
from datetime import timedelta
import requests
import time
from time import sleep
from json import dumps
from kafka import KafkaProducer
from kafka import KafkaConsumer
from json import loads
import os
import sqlalchemy as db
from sqlalchemy import create_engine, Column, Integer, String, Sequence, Float,PrimaryKeyConstraint, ForeignKey
from sqlalchemy.schema import CreateSchema
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, backref
from sqlalchemy.sql import *

In [2]:
df_jams = pd.read_csv("dataset/jams.csv")
mainData = []
for i in range(len(df_jams)):
    mainData.append(df_jams.iloc[i].to_dict())

In [3]:
# Kafka & Power BI Streaming
### filter keys that used ro visualization 
def filter_key(jamsdata):
    key_used = ['city','delay','length','level','speedKMH','peak']
    data =  {k: v for k, v in jamsdata.items() if k in key_used}
    data['real_time'] = dt.datetime.now().isoformat()
    return data

### format data to accepted json power BI
def data_to_push(data):
    HEADER = ['city','delay','real_time','length','level','speedKMH','peak']
    data['speedKMH'] = round(data['speedKMH'],1)
    data_to_push = [data]
    data_df = pd.DataFrame(data_to_push, columns=HEADER)
    data_json = bytes(data_df.to_json(orient='records'), encoding='utf-8')
    return data_json

def publish_message(producer_instance, topic_name, value):
    try:
        producer_instance.send(topic_name, value=value)
        producer_instance.flush()
        print('Message published successfully.')
    except Exception as ex:
        print('Exception in publishing message')
        print(str(ex))

def connect_kafka_producer(ip,host):
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['%s:%s' % (ip,host)], api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [5]:
# Kafka & Power BI
# if __name__ == '__main__':
rawdata = mainData
data_to_stream = rawdata
ip = '35.236.82.108'
host = '9092'
kafka_producer = connect_kafka_producer(ip,host)
i=0
REST_API_URL = 'https://api.powerbi.com/beta/678e01f9-d83a-4406-92dd-58b7e9b78206/datasets/75d72cdc-a9ab-422d-b646-396319aacee5/rows?redirectedFromSignup=1&key=KLdtYk9zWfgVEN3ptvtMCTBs3vIdmkpdEUVy820KjteQmCa9VSv6VE2cmo0%2FQlO4yOXh9BWdHdCQAD2%2BWRgJGA%3D%3D'
while i < len(data_to_stream):
    try:
        data_to_push_kafka = filter_key(data_to_stream[i])
        data_json = data_to_push(data_to_push_kafka)
        print("JSON dataset", data_json)

        # Push the data to kafka
        publish_message(kafka_producer, 'NovindraPrasetio-finaltest-kafka', data_json)
#         print("Data has ben pushed to kafka")

        # Post the data on the Power BI API
        req = requests.post(REST_API_URL, data_json)
#         print("Data posted in Power BI API")

        time.sleep(2)
        i +=1
    except:
        break

JSON dataset b'[{"city":"Bogor","delay":160,"real_time":"2019-12-13T09:51:21.713908","length":957,"level":2,"speedKMH":11.5,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":190,"real_time":"2019-12-13T09:51:25.390770","length":290,"level":4,"speedKMH":4.3,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":173,"real_time":"2019-12-13T09:51:27.985751","length":771,"level":3,"speedKMH":11.1,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":245,"real_time":"2019-12-13T09:51:30.549158","length":877,"level":3,"speedKMH":9.5,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":162,"real_time":"2019-12-13T09:51:33.181916","length":2005,"level":2,"speedKMH":21.7,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":122,"real_time":"2019-12-13T09:51:35.825944","length":414,"level":3,"speedKMH":8.0,"peak":0}]'
Message published successf

JSON dataset b'[{"city":"Depok","delay":205,"real_time":"2019-12-13T09:53:21.890776","length":1129,"level":3,"speedKMH":13.2,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":130,"real_time":"2019-12-13T09:53:24.247818","length":1233,"level":2,"speedKMH":17.8,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":123,"real_time":"2019-12-13T09:53:26.677468","length":321,"level":3,"speedKMH":7.6,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":1299,"real_time":"2019-12-13T09:53:29.091692","length":2146,"level":4,"speedKMH":4.8,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":234,"real_time":"2019-12-13T09:53:31.473386","length":236,"level":4,"speedKMH":3.2,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":165,"real_time":"2019-12-13T09:53:33.905349","length":564,"level":3,"speedKMH":7.2,"peak":0}]'
Message published succes

Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":276,"real_time":"2019-12-13T09:55:31.775042","length":348,"level":3,"speedKMH":3.6,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":128,"real_time":"2019-12-13T09:55:34.223838","length":251,"level":4,"speedKMH":6.2,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":170,"real_time":"2019-12-13T09:55:36.615470","length":505,"level":3,"speedKMH":7.9,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":683,"real_time":"2019-12-13T09:55:39.017265","length":1714,"level":3,"speedKMH":7.2,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":425,"real_time":"2019-12-13T09:55:41.481974","length":1003,"level":3,"speedKMH":6.8,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":283,"real_time":"2019-12-13T09:55:43.909692","length":348,"level":4,"speedKMH":3.5,"peak":

Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":175,"real_time":"2019-12-13T09:57:34.578236","length":301,"level":3,"speedKMH":4.9,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":307,"real_time":"2019-12-13T09:57:37.212790","length":989,"level":3,"speedKMH":9.2,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":254,"real_time":"2019-12-13T09:57:40.059354","length":416,"level":4,"speedKMH":4.7,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":248,"real_time":"2019-12-13T09:57:42.677117","length":1467,"level":2,"speedKMH":12.5,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":311,"real_time":"2019-12-13T09:57:45.501554","length":1692,"level":3,"speedKMH":11.9,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":128,"real_time":"2019-12-13T09:57:48.161054","length":366,"level":4,"speedKMH":8.5,

Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":164,"real_time":"2019-12-13T09:59:41.256900","length":1076,"level":2,"speedKMH":13.6,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Karawang","delay":148,"real_time":"2019-12-13T09:59:43.682252","length":507,"level":3,"speedKMH":9.1,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":574,"real_time":"2019-12-13T09:59:46.127338","length":1070,"level":4,"speedKMH":5.6,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":305,"real_time":"2019-12-13T09:59:48.557419","length":449,"level":4,"speedKMH":4.6,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":118,"real_time":"2019-12-13T09:59:51.002446","length":1190,"level":2,"speedKMH":19.1,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":149,"real_time":"2019-12-13T09:59:53.337761","length":2357,"level":1,"speedKMH":24.6

...

JSON dataset b'[{"city":"Bekasi","delay":119,"real_time":"2019-12-13T11:02:56.349454","length":488,"level":2,"speedKMH":8.1,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":160,"real_time":"2019-12-13T11:02:58.971868","length":576,"level":3,"speedKMH":8.8,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":567,"real_time":"2019-12-13T11:03:01.400750","length":537,"level":4,"speedKMH":3.0,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":155,"real_time":"2019-12-13T11:03:03.834115","length":808,"level":3,"speedKMH":12.1,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":152,"real_time":"2019-12-13T11:03:06.531138","length":2180,"level":2,"speedKMH":20.9,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":158,"real_time":"2019-12-13T11:03:09.365426","length":860,"level":2,"speedKMH":12.2,"peak":0}]'
Message published succes

JSON dataset b'[{"city":"Bekasi","delay":325,"real_time":"2019-12-13T11:05:00.152400","length":1279,"level":3,"speedKMH":9.9,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":124,"real_time":"2019-12-13T11:05:02.547772","length":410,"level":3,"speedKMH":8.1,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":949,"real_time":"2019-12-13T11:05:04.967499","length":986,"level":4,"speedKMH":3.1,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":189,"real_time":"2019-12-13T11:05:07.392737","length":1369,"level":3,"speedKMH":14.6,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":164,"real_time":"2019-12-13T11:05:09.784397","length":1076,"level":2,"speedKMH":13.6,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":162,"real_time":"2019-12-13T11:05:12.163525","length":1771,"level":2,"speedKMH":16.6,"peak":0}]'
Message published

Message published successfully.
JSON dataset b'[{"city":"Karawang","delay":246,"real_time":"2019-12-13T11:07:08.681542","length":1141,"level":3,"speedKMH":11.7,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Karawang","delay":153,"real_time":"2019-12-13T11:07:11.385111","length":1695,"level":2,"speedKMH":19.9,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":190,"real_time":"2019-12-13T11:07:14.130670","length":290,"level":4,"speedKMH":4.3,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":160,"real_time":"2019-12-13T11:07:16.851159","length":957,"level":2,"speedKMH":11.5,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":173,"real_time":"2019-12-13T11:07:19.481760","length":771,"level":3,"speedKMH":11.1,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":245,"real_time":"2019-12-13T11:07:21.922247","length":877,"level":3,"speedKMH":9.

Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":162,"real_time":"2019-12-13T11:09:12.290405","length":2005,"level":2,"speedKMH":21.8,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":121,"real_time":"2019-12-13T11:09:14.920455","length":414,"level":3,"speedKMH":8.1,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":191,"real_time":"2019-12-13T11:09:17.623832","length":713,"level":3,"speedKMH":9.8,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":142,"real_time":"2019-12-13T11:09:20.278159","length":2076,"level":2,"speedKMH":21.2,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":230,"real_time":"2019-12-13T11:09:22.923201","length":825,"level":3,"speedKMH":9.4,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Bekasi","delay":206,"real_time":"2019-12-13T11:09:25.589706","length":1477,"level":2,"speedKMH":13.9,"pe

JSON dataset b'[{"city":"Depok","delay":828,"real_time":"2019-12-13T11:11:22.598332","length":1542,"level":4,"speedKMH":5.3,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":392,"real_time":"2019-12-13T11:11:25.201912","length":511,"level":3,"speedKMH":3.7,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":217,"real_time":"2019-12-13T11:11:27.865606","length":574,"level":3,"speedKMH":6.8,"peak":1}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":129,"real_time":"2019-12-13T11:11:30.509035","length":546,"level":3,"speedKMH":10.7,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Depok","delay":131,"real_time":"2019-12-13T11:11:33.087701","length":374,"level":3,"speedKMH":7.3,"peak":0}]'
Message published successfully.
JSON dataset b'[{"city":"Bogor","delay":574,"real_time":"2019-12-13T11:11:35.668172","length":1070,"level":4,"speedKMH":5.6,"peak":1}]'
Message published successfu

### Batching Process

In [51]:
## Making class
class Maker(object):
    def read_file(self, table_name):
        return pd.read_csv(table_name).sample(n=100, random_state = 100, replace=False)
    def connect_db(DB_USER, DB_PASS, DB_IP, DB_PORT, DB_NAME):
        # create db create_engine
        dbs = create_engine(f'postgresql://{DB_USER}:{DB_PASS}@{DB_IP}:{DB_PORT}/{DB_NAME}')
        connection = dbs.connect()
        metadata = db.MetaData(schema="test")
        return dbs
    
maker = Maker()

In [52]:
# # Read the part of "Home Creditin"
# df_jams_batch = maker.read_file("dataset/jams.csv")
# df_jams_batch.head()

In [53]:
df_jams_batch = df_jams.head(999)

In [54]:
db_user = "iykra_trainee"
db_pass = "passwordiykra"
db_name = "postgres"
db_ip = "157.230.240.246"
db_port = "5432"

Base = declarative_base()

In [60]:
dbs = Maker.connect_db(db_user, db_pass, db_ip, db_port, db_name)

In [62]:
df_jams_batch.to_sql('novindraprasetio_waze_df', dbs, schema='test',if_exists='replace', index=False)